# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector
from Queries.TPCH_SF1_ORDERS import *

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def query_cursor(wh_name: str, db_name: str, schema_name: str):
    '''
    Create a cursor for interacting with database

    Parameters

    wh_name: str
      warehouse name
    
    db_name: str
      database name

    schema_name: str
      schema name

    Returns:
    The cursor itself, or None if some error happened, or the response returned by Snowflake if the _no_results flag is on.
    
    '''
    cnx = snowflake.connector.connect(user=USERNAME, password=PASSWORD, account=ACCOUNT_ID)

    cur = cnx.cursor().execute(f'USE WAREHOUSE {wh_name}')
    cur = cnx.cursor().execute(f'USE DATABASE {db_name}')
    cur = cnx.cursor().execute(f'USE SCHEMA {schema_name}')

    return cur


def fetch_data(wh_name, db_name, schema_name, sql_query, col_names) -> pd.DataFrame:
    '''
    Retrieves data from a Snowflake database and returns it as a Pandas DataFrame.

    Parameters

    wh_name: str
      warehouse name
    
    db_name: str
      database name

    schema_name: str
      schema name

    sql_query: str
        query to execute

    col_names: list[str]
        column names to use

    Returns:
        result as pandas dataframe
    
    '''
    cur = query_cursor(wh_name=wh_name, db_name=db_name, schema_name=schema_name)
    print(f'Snowflake Query ID: {cur.sfqid}')
    results = cur.execute(f'{sql_query}').fetchall()
    
    return pd.DataFrame(results, columns=col_names)

# Create Connection To Warehouse, Database and Schema

In [3]:
cur = query_cursor(WAREHOUSE, DATABASE, SCHEMA)

# Query Data

In [4]:
column_names = ['orderkey', 'custkey', 'orderstatus', 'totalprice', 'orderdate', 'orderpriority']

df = fetch_data(WAREHOUSE, DATABASE, SCHEMA, query1, column_names)

Snowflake Query ID: 01adab8d-3201-dca3-0002-0ed200012ff2


In [5]:
df

,orderkey,custkey,orderstatus,totalprice,orderdate,orderpriority
0,66,129200,F,103740.67,1994-01-20,5-LOW
1,70,64340,F,113534.42,1993-12-18,5-LOW
2,96,107779,F,68989.90,1994-04-17,2-HIGH
3,167,119402,F,71124.35,1993-01-04,4-NOT SPECIFIED
4,423,103396,O,50240.88,1996-06-01,1-URGENT
...,...,...,...,...,...,...
1499995,2999908,23014,F,168982.73,1994-04-30,5-LOW
1499996,2999939,99310,O,260146.08,1998-05-16,5-LOW
1499997,2999975,112270,O,46122.59,1995-06-07,5-LOW
1499998,2998694,34436,O,83606.81,1995-11-12,5-LOW
